# Multi Agent-Systems
A team is a group of agents that work together to achieve a common goal

In [12]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage
from autogen_agentchat.teams import RoundRobinGroupChat

from pydantic import BaseModel
from PIL import Image
from io import BytesIO
import requests

from IPython.display import Markdown, display
from dotenv import load_dotenv
import os

load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
model_name = os.getenv("GEMINI_MODEL")

### Model Client

In [8]:
model_client = OpenAIChatCompletionClient(
    model=model_name,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=google_api_key,
    model_info={
        "vision": True,
        "function_calling": True,
        "json_output": True,
        "family": "gpt-4o",
        "structured_output": True
    },
)

### Single Agent Approach
A single agent is going to create a short story for us

In [9]:
story_agent = AssistantAgent(
    name='story_agent',
    model_client=model_client,
    system_message='You are a creating writer. Generate a short story about a brave knight and a dragon.'
)

In [10]:
async def test_simple_agent():
    task = TextMessage(
        content='Write a short story about a brave knight and a dragon. Keep it up to 50 words',
        source='user'
    )
    result = await story_agent.run(task=task)
    # print(result.messages[-1].content)
    display(Markdown(result.messages[-1].content))

await test_simple_agent()

Sir Kaelen’s armor sizzled. He dodged a torrent of fire, his shield held high. The dragon roared, a sound of thunder and rage. With a final, desperate lunge, the knight’s enchanted sword found its mark. The great beast fell, and the kingdom was finally safe.

### Multi Agent Team Approach
We will be using 3 agents to now write the story, each with its own work and mastery

In [11]:
plot_agent = AssistantAgent(
    name = 'plot_writer',
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on the Pokemon's journey"
)

character_agent = AssistantAgent(
    name = 'character_writer',
    model_client=model_client,
    system_message="You develop characters. Describe the pokemon and the villian in detail, including their motivations and backgrounds."
)

ending_agent = AssistantAgent(
    name = 'ending_writer',
    model_client=model_client,
    system_message="You wrute engaging endings. conclude the story with a twist."
)

`RoundRobinGroupChat` is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

In [16]:
team = RoundRobinGroupChat(
    participants= [plot_agent, character_agent, ending_agent],
    max_turns=3
)
team.description

'A team of agents.'

In [17]:
async def test_team():
    task = TextMessage(
        content='Write a short story a brave boy and his Pokemon. Keep it up to 50 words',
        source='user'
    )

    result = await team.run(task=task)
    for each_agent_message in result.messages:
        print(f'{each_agent_message.source} : {each_agent_message.content}')
        

await test_team()

user : Write a short story a brave boy and his Pokemon. Keep it up to 50 words
plot_writer : The cave entrance was blocked by a snarling Mightyena. My trainer, Leo, froze. He was scared, but I wasn't. I was his Riolu. I leaped forward, paws glowing with a new power. My first Force Palm struck true, clearing our path. My journey to protect him had begun.
character_writer : Based on your story, here are the detailed character developments:

### The Pokémon: Riolu

**Background:** This Riolu was hatched from an egg given to Leo by his parents, who hoped it would help their timid son grow more confident. From the moment it hatched, Riolu sensed Leo’s gentle nature and his underlying anxiety. It never saw Leo’s fear as a weakness, but as something precious that needed protecting. It has spent its short life watching Leo, learning his habits, and feeling a powerful, instinctual bond that goes beyond typical training.

**Motivation:** Riolu's motivation is one of pure, unwavering loyalty. It 